In [1]:
!pip install dfply
!pip install plotly
!pip install xhtml2pdf
import pandas as pd
import numpy as np
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statistics 
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import pearsonr
import plotly.graph_objects as go
import warnings
from scipy.stats import shapiro
from datetime import date, timedelta

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
sidelined = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/sidelined_Clean.pkl")

In [0]:
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")
players = players[players.sofifa_id.notnull()]
players.sofifa_id = players.sofifa_id.astype("int") 

In [0]:
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_detailed.pkl")

In [0]:
sidelined = sidelined.drop_duplicates() 

In [0]:
sidelined.reset_index(inplace = True, drop = True)

In [0]:
sidelined = sidelined[sidelined.sofifa_id.isin(players.sofifa_id.unique())]

In [0]:
sidelined = pd.merge(sidelined, players[["tm_id","sofifa_id"]], on = "sofifa_id")

In [15]:
sidelined = sidelined[sidelined.active == False]
sidelined["year"] = sidelined["end_date"].apply(lambda x : x.year)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
sidelined = pd.merge(sidelined, markval, on = ["tm_id","year"])

In [0]:
dur = sidelined.duration
sidelined = sidelined[sidelined.duration.between(dur.quantile(0.15), dur.quantile(0.85))]

In [35]:
corr, _ = pearsonr(sidelined["duration"],sidelined["mv"])
print(corr)
fig = px.scatter(sidelined, "duration","mv",labels = {"duration":"", "mv":""})
fig.show()

0.0430562508485859


The variables are not correlated.

In [0]:
sidelined = sidelined.groupby("tm_id").apply(lambda x : x.sort_values("year"))

In [0]:
sidelined.drop_level("tm_id")

In [0]:
sidelined = sidelined.reset_index(drop = True)

In [0]:
sidelined["cum_missed_days"] = sidelined.groupby("tm_id").apply(lambda x : x["duration"].cumsum()).values

In [51]:
corr, _ = pearsonr(sidelined["duration"],sidelined["mv"])
print(corr)
fig = px.scatter(sidelined, "cum_missed_days","mv",labels = {"cum_missed_days":"Number of missed days", 
                                                             "mv":"Market Value"})
fig.show()

0.043056250848585914


In [0]:
# May be connected to stats